# ISO TC/211 Harmonised Model Controls

Connect to the EA app and model repository

In [1]:
from Parameters import *
from EAConnect import *
from HM_Controls import *
import sys
import pandas as pd

# Open EA Repository and find Model
eaApp = openEAapp()
eaRepo = openEArepo(eaApp,repo_path)
try:
    omMod = eaRepo.Models.GetByName(modelName)
    printTS('Model "' + modelName + '" found with PackageGUID ' + omMod.PackageGUID )
except Exception as e:
    printTS('Model  "' + modelName + '" not found!')
    closeEA(eaRepo)
    sys.exit()
printTS('Number of main packages: ' + str(omMod.Packages.Count))
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

2025-01-03 16:51:15   Hi EA - are you there? 
2025-01-03 16:51:15   I am here
2025-01-03 16:51:15   Hi EA - Please open this repository: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\ISOTC211_HM EditorialVersion.qea
2025-01-03 16:51:18   OK! Repository C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\ISOTC211_HM EditorialVersion.qea is ready!
2025-01-03 16:51:18   Model "Conceptual Models" found with PackageGUID {7B6B28E9-C583-4363-9E9C-F37A37AE06C9}
2025-01-03 16:51:18   Number of main packages: 5


## Select a package in EA before continuing

In [2]:
thePackage = eaRepo.GetTreeSelectedPackage()
printTS('Selected package name: ' + thePackage.Name)

2025-01-03 16:51:31   Selected package name: ISO/CD 19130-2 Edition 2


**Package tags and alias**

Add or update package tags and alias

In [11]:
lstMd = {}
lstMd['name'] = 'Imagery sensor models for geopositioning - Part 2: SAR, InSAR, lidar and sonar'
lstMd['number'] = '19130-2'
lstMd['edition'] = '2'
lstMd['publicationDate'] = ''
lstMd['yearVersion']= '2024'

eaEl = thePackage.Element

#Update existing tags and add missing tags
for key, value in lstMd.items():
    print(f"{key}: {value}")
    try:
        eaTag = eaEl.TaggedValues.GetByName(key)
        if not eaTag is None:
            eaTag.Value = value
            eaTag.Update()
        else:
            print('New tag!')
            eaTag = eaEl.TaggedValues.AddNew(key,value)
            eaTag.Update()    
    except:
        print('Something went wrong!')
eaEl.TaggedValues.Refresh()

#Set alias
eaEl.Alias = lstMd['name']
eaEl.Update()

#TODO:Update status?



name: Imagery sensor models for geopositioning - Part 2: SAR, InSAR, lidar and sonar
New tag!
number: 19130-2
New tag!
edition: 2
New tag!
publicationDate: 
New tag!
yearVersion: 2024
New tag!


True

**Duplicates:**

Check that there are no duplicate element names

In [3]:
df = duplicateElements(thePackage)
non_unique = df[df.duplicated(subset=['ElementName'], keep=False)]
errCount = len(non_unique)
printTS('Number of errors: ' + str(errCount))

if errCount > 0:
    print('')
    print('Duplicate elements: ')
    for index, row in non_unique.iterrows():
        combined_string = f"{row['ElementName']}"
        print(combined_string)

2025-01-03 16:28:58   ----------------------------
2025-01-03 16:28:58   Package: Sensor Data Extension
2025-01-03 16:28:58   ERROR|Duplicate element: Antenna
2025-01-03 16:28:58   ERROR|Duplicate element: Antenna
2025-01-03 16:28:58   ERROR|Duplicate element: Collection
2025-01-03 16:28:58   ERROR|Duplicate element: CoordinateFrame
2025-01-03 16:28:58   ERROR|Duplicate element: Pattern
2025-01-03 16:28:58   ERROR|Duplicate element: PhaseCenter
2025-01-03 16:28:58   ERROR|Duplicate element: Product
2025-01-03 16:28:58   ERROR|Duplicate element: Scene
2025-01-03 16:28:58   ERROR|Duplicate element: SD_Dynamics
2025-01-03 16:28:58   ERROR|Duplicate element: Steerable
2025-01-03 16:28:58   ERROR|Duplicate element: TransmitReceive
2025-01-03 16:28:58   ERROR|Duplicate element: Unsteerable
2025-01-03 16:28:58   ----------------------------
2025-01-03 16:28:58   Package: AT
2025-01-03 16:28:59   ERROR|Duplicate element: SD_SensorSystemAndOperation
2025-01-03 16:28:59   ERROR|Duplicate element

**Data type references:**

Check that all data types are references to an element.
Missing references to primitive data types are fixed in the script.

The results are stored in a data frame for further use.

In [4]:
#df = pd.DataFrame(columns=['FullPath','Package','Element','Property','DependentPackage','DependentElement','GUID'])
df = listClassifiers(eaRepo,thePackage)
noRef = df[df['GUID'].isna()]
errCount = len(noRef)
printTS('')
printTS('Number of errors: ' + str(errCount))

print('')
print('Attributes without reference:')
for index, row in noRef.iterrows():
    combined_string = f"{row['Package']}.{row['Element']}.{row['Property']} (Data type:{row['DependentElement']})"
    print(combined_string)




2025-01-03 16:29:05   ----------------------------
2025-01-03 16:29:05   Package: Sensor Data Extension
2025-01-03 16:29:05   INFO|Fixing referenced element for attribute: ISO TC211.ISO 19130 Imagery sensor models for geopositioning.ISO/CD 19130-2 Edition 2.Integer (GUID = {E6A6E360-E994-45c5-8B4A-E95E2E3553E1})
2025-01-03 16:29:06   ERROR|Missing data type connection for attribute :KSpace.kRowWindowParameters (Data type: Dictionary)
2025-01-03 16:29:06   ERROR|Missing data type connection for attribute :KSpace.kColWindowParameters (Data type: Dictionary)
2025-01-03 16:29:06   ERROR|Missing data type connection for attribute :Polarization.electronicBoresight (Data type: Dictionary)
2025-01-03 16:29:07   ERROR|Missing data type connection for attribute :Processing.polCalParameters (Data type: Dictionary)
2025-01-03 16:29:07   ERROR|Missing data type connection for attribute :Processing.nonstandardParameters (Data type: Dictionary)
2025-01-03 16:29:07   ERROR|Missing data type connection

**Dependencies:**

Check which packages data types and relations are connected to.

In [5]:
dfCounts = df.groupby(['DependentPackage']).size()
print(dfCounts)

DependentPackage
ISO TC211.ISO 19103 Conceptual schema language.ISO 19103 Edition 2.Core Data Types                             591
ISO TC211.ISO 19130 Imagery sensor models for geopositioning.ISO/CD 19130-2 Edition 2.AT                        39
ISO TC211.ISO 19130 Imagery sensor models for geopositioning.ISO/CD 19130-2 Edition 2.AT2                       65
ISO TC211.ISO 19130 Imagery sensor models for geopositioning.ISO/CD 19130-2 Edition 2.SAR                      126
ISO TC211.ISO 19130 Imagery sensor models for geopositioning.ISO/CD 19130-2 Edition 2.Sensor Data Extension     84
ISO TC211.ISO 19130 Imagery sensor models for geopositioning.ISO/CD 19130-2 Edition 2.lidar                     28
ISO TC211.ISO 19130 Imagery sensor models for geopositioning.ISO/CD 19130-2 Edition 2.sonar                     28
dtype: int64


**Dependencies:**

List elements from a specific package 

In [8]:
strFilter = 'ISO TC211.ISO 19115 Metadata'

df_cleaned = df.dropna(subset=['DependentPackage'])
filtered_df = df_cleaned[df_cleaned['DependentPackage'].str.startswith(strFilter)]

if len(filtered_df) > 0:
    print('Refered elements from package "' + strFilter + '*"')
    for index, row in filtered_df.iterrows():
        print(f"{row['Element']}.{row['Property']} (Data type:{row['DependentPackage']}.{row['DependentElement']})")
else:
    print('No elements found!')    

Refered elements from package "ISO TC211.ISO 19115 Metadata*"
ValueAssignment.Dependency (Data type:ISO TC211.ISO 19115 Metadata.ISO 19115-2 Edition 2 (Complete).Acquisition information.MI_Event)
ValueAssignment.Dependency (Data type:ISO TC211.ISO 19115 Metadata.ISO 19115-1 Edition 1 (Complete).Lineage information.LI_ProcessStep)


**Upgrade model to new 19103**

Refer attribute data types to core types from 19103 Edition 2
Update classifiers to type and stereotype from 19103 Edition 2

In [ ]:
dfCSL = fixCSL(eaRepo,thePackage)

**Definitions:**

Check that all classes, attributes and navigable associations have definitions

In [6]:
#defDf = pd.DataFrame(columns=['Type','PackageName','ElementName','PropertyName','Supplier'])
defDf = listMissingDefinitions(eaRepo,thePackage)

errCount = len(defDf)
printTS('')
printTS('Number of errors: ' + str(errCount))
print('')


noDef = defDf[defDf['PropertyName'].isna() & defDf['Supplier'].isna()]
if len(noDef) > 0:
    print('')
    print('Elements without definitions (' + str(len(noDef)) + '):')
    for index, row in noDef.iterrows():
        combined_string = f"{row['PackageName']}.{row['ElementName']}"
        print(combined_string)

noDef = defDf[defDf['Type']=='Attribute']
if len(noDef) > 0:
    print('')
    print('Attributes without definitions (' + str(len(noDef)) + '):')
    for index, row in noDef.iterrows():
        combined_string = f"{row['PackageName']}.{row['ElementName']}.{row['PropertyName']}"
        print(combined_string)

noDef = defDf[defDf['Type']=='Code value']
if len(noDef) > 0:
    print('')
    print('Code values without definitions (' + str(len(noDef)) + '):')
    for index, row in noDef.iterrows():
        combined_string = f"{row['PackageName']}.{row['ElementName']}.{row['PropertyName']}"
        print(combined_string)

noDef = defDf[defDf['Type']=='Role name']
if len(noDef) > 0:
    print('')
    print('Navigable association ends without role name (' + str(len(noDef)) + '):')
    for index, row in noDef.iterrows():
        combined_string = f"{row['PackageName']}.{row['ElementName']} towards {row['Supplier']}"
        print(combined_string)

noDef = defDf[defDf['Type']=='Role']
if len(noDef) > 0:
    print('')
    print('Navigable association ends without definition (' + str(len(noDef)) + '):')
    for index, row in noDef.iterrows():
        if row['PropertyName'] != None:
            combined_string = f"{row['PackageName']}.{row['ElementName']}.{row['PropertyName']} towards {row['Supplier']}"
        else:
            combined_string = f"{row['PackageName']}.{row['ElementName']} towards {row['Supplier']}"
        print(combined_string)


2025-01-03 16:29:39   ----------------------------
2025-01-03 16:29:39   Package: Sensor Data Extension
2025-01-03 16:29:39   ERROR|Missing definition for:Class in package Sensor Data Extension:Antenna
2025-01-03 16:29:39   ERROR|Missing definition for:Class in package Sensor Data Extension:Antenna
2025-01-03 16:29:39   ERROR|Missing definition for attribute in package Sensor Data Extension: Antenna.transmitPattern
2025-01-03 16:29:39   ERROR|Missing definition for attribute in package Sensor Data Extension: Antenna.transmitPhaseCenter
2025-01-03 16:29:39   ERROR|Missing definition for attribute in package Sensor Data Extension: Antenna.transmitElectronicBoresight
2025-01-03 16:29:39   ERROR|Missing definition for attribute in package Sensor Data Extension: Antenna.receivePattern
2025-01-03 16:29:39   ERROR|Missing definition for attribute in package Sensor Data Extension: Antenna.receivePhaseCenter
2025-01-03 16:29:39   ERROR|Missing definition for attribute in package Sensor Data Ext

**Diagram representation:**

Check that all elements are shown in at least one diagram

In [3]:
dfE = elementsInDiagrams(thePackage)
errCount = len(dfE)
printTS('')
printTS('Number of errors: ' + str(errCount))

if errCount > 0:
    print('')
    print('Elements that are not in any diagram:')
    for index, row in dfE.iterrows():
        combined_string = f"{row['PackageName']}.{row['ElementName']}"
        print(combined_string)    


2025-01-03 16:51:43   ----------------------------
2025-01-03 16:51:43   Package: Sensor Data Extension
2025-01-03 16:51:43   Diagram: Original (36 objects)
2025-01-03 16:51:43   Diagram: Figure 1 — Extension of the Sensor Model (3 objects)
2025-01-03 16:51:43   Diagram: Figure 2 — SE_Dynamics, extension of SD_Dynamics (2 objects)
2025-01-03 16:51:43   Diagram: Figure 3 — SE_PlatformDynamics, extension of SD_PlatformDynamics (3 objects)
2025-01-03 16:51:43   ----------------------------
2025-01-03 16:51:43   Package: AT
2025-01-03 16:51:43   Diagram: Figure 19 — Aerial triangulation, general (15 objects)
2025-01-03 16:51:43   ----------------------------
2025-01-03 16:51:43   Package: AT2
2025-01-03 16:51:43   Diagram: Original (15 objects)
2025-01-03 16:51:43   Diagram: Figure 20 — Aerial triangulation, observations (9 objects)
2025-01-03 16:51:43   Diagram: Figure 21 — Other result of aerial triangulation (6 objects)
2025-01-03 16:51:43   ----------------------------
2025-01-03 16:51

In [ ]:
# Delete elements that are not in any diagram
for index, row in dfE.iterrows():
    combined_string = f"{row['PackageName']}.{row['ElementName']} ({row['GUID']})"
    #print(combined_string)    
    elGUID = row['GUID']
    el = eaRepo.GetElementByGuid(elGUID)
    #Delete the element
    if el:
            # Delete the element
            eaRepo.DeleteElement(el.ElementID)
            # Refresh the model view
            # eaRepo.RefreshModelView(0)
            print(f"Element {el} deleted successfully.")
    else:
        print(f"Element {el} not found.")

{1E1747FA-2285-4fbd-83B0-DCA8164C0A8E}
{F90DF245-868B-4eb1-A22F-5E4FFACBFC0E}
{A5315CEB-462B-49d1-A5CA-EF01783B9056}
{FA569FA7-E6C6-485b-8166-60081CC26C44}
{7098CB00-EB36-4647-8C3A-9894F576E3F5}
{5493B2EC-EC02-41b2-BE94-4A0053B3289F}
{53C13FC1-2752-4182-A096-17C46F367100}
{5ACDC51D-CE3B-436d-A3B7-F148722C69C1}
{5FF44588-EE63-4cd9-960D-BC6C293F4197}
{6E21FD1E-B0DD-4e80-A3D5-4A9DD3A361C2}
{3E4FBE9E-DFF6-40d7-84B4-6F35D8D438B3}
{3592FD1C-1ABD-4913-B6B3-1B7905DCF29F}
{7BB9E439-D1A8-4a97-8DE9-C5480ACF7707}
{6CC03DC2-A0DF-47fa-A36E-1B607065DBE1}
{F6F1B4FD-640F-4385-9C2A-2D07F0A59EEB}
{04466EE2-1623-4098-B3A9-891D89A4DC84}
{48161338-FADB-4e48-8B75-DA6B79E95AE0}
{17B98566-6E26-487d-A261-269F0BC7D291}
{66008EF7-6022-4f90-A2A2-3E170E442B38}
{9CF38C67-7096-F437-A5B0-7DA44177A36B}
{8D0E8C11-10D4-2F42-B792-0998EC0BA51A}
{960DBC6C-293F-4197-BE34-A8326CFFE8EF}
{BAC4C19B-7FD1-E95E-A459-C434E6C59071}
{9D544D62-2228-8C1C-A653-C26D8FB9ACBF}
{B32356F0-0D47-2463-BF47-197A6263CDE6}
{AF3CC093-3177-D464-AD57-

**Diagram layout:**

Setting diagram fonts to Cambria and hiding "isSubstitutable" labels

In [19]:
recDiagramCleaning(thePackage)

2025-01-03 14:14:01   Diagram count: 0
2025-01-03 14:14:01   ----------------------------
2025-01-03 14:14:01   Package: Sensor Data Extension
2025-01-03 14:14:01   Diagram count: 1
2025-01-03 14:14:01   Diagram: d1 (36 objects)
2025-01-03 14:14:01   ----------------------------
2025-01-03 14:14:01   Package: AT
2025-01-03 14:14:01   Diagram count: 1
2025-01-03 14:14:01   Diagram: d1 (15 objects)
2025-01-03 14:14:01   ----------------------------
2025-01-03 14:14:01   Package: AT2
2025-01-03 14:14:01   Diagram count: 1
2025-01-03 14:14:01   Diagram: d1 (15 objects)
2025-01-03 14:14:01   ----------------------------
2025-01-03 14:14:01   Package: lidar
2025-01-03 14:14:01   Diagram count: 1
2025-01-03 14:14:01   Diagram: d1 (15 objects)
2025-01-03 14:14:01   ----------------------------
2025-01-03 14:14:01   Package: InSAR
2025-01-03 14:14:01   Diagram count: 1
2025-01-03 14:14:01   Diagram: d1 (61 objects)
2025-01-03 14:14:01   ----------------------------
2025-01-03 14:14:01   Package

Export error report

In [8]:
print('')
file_path = mainFolder + '\\Controls.xlsx'
# Export to Excel 
writer = pd.ExcelWriter(file_path)
noDef = df[df['DependentPackage'].isna() | (df['DependentPackage'] == '')]
if len(noDef) > 0:
    noDef.to_excel(writer,'Missing datatype references') 
    print(f"Exported datatype report to file: {file_path}") 

noDef = defDf[defDf['Type']!='Role name']
if len(noDef) > 0:
    noDef.to_excel(writer,'Missing definitions') 
    print(f"Exported definitions report to file: {file_path}") 

noDef = defDf[defDf['Type']=='Role name']
if len(noDef) > 0:
    noDef.to_excel(writer,'Missing role names') 
    print(f"Exported role names report to file: {file_path}") 

if len(non_unique) > 0:
    non_unique.to_excel(writer,'Non unique names') 
    print(f"Exported duplication report to file: {file_path}") 

if len(dfE) > 0:
    dfE.to_excel(writer,'Not in diagram') 
    print(f"Exported diagram report to file: {file_path}")     

writer.close()



Exported datatype report to file: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\Controls.xlsx
Exported definitions report to file: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\Controls.xlsx
Exported role names report to file: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\Controls.xlsx
Exported duplication report to file: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\Controls.xlsx
Exported diagram report to file: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\Controls.xlsx


C:\Users\JETKNU\AppData\Local\Temp\ipykernel_18196\1846826498.py:7: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  noDef.to_excel(writer,'Missing datatype references')
C:\Users\JETKNU\AppData\Local\Temp\ipykernel_18196\1846826498.py:12: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  noDef.to_excel(writer,'Missing definitions')
C:\Users\JETKNU\AppData\Local\Temp\ipykernel_18196\1846826498.py:17: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  noDef.to_excel(writer,'Missing role names')
C:\Users\JETKNU\AppData\Local\Temp\ipykernel_18196\1846826498.py:21: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  non_unique.to_excel(writer,'Non uni

**Export to XMI:**

Export the package to XMI for upload to GitHub

In [12]:
# Replace colons (":") with underscores ("_") in the filename
fName = thePackage.Name.replace(":", "_")
# Replace forward slashes ("/") with an empty string
fName = fName.replace("/", "")
# Combine the modified filename with the path and add the ".xml" extension
fName = xmiPath + fName + ".xml"

thePackage.IsControlled = -1
thePackage.XMLPath = fName
thePackage.BatchSave = 1
thePackage.BatchLoad = 1
thePackage.Update

#XmiExportType = 3
pI = eaRepo.GetProjectInterface()
result = pI.ExportPackageXMI(thePackage.PackageGUID, 3, 1, -1, 1, 0, fName)
print(result)